# Install necessary packages

In [15]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading

Load the dataset.

In [2]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    data = pd.read_csv(file_path)
    return data  

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [3]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    col_data = df[[column_to_encode]]

    encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap
    encoded_array = encoder.fit_transform(col_data)

    encoded_col_names = encoder.get_feature_names_out([column_to_encode])

    encoded_df = pd.DataFrame(encoded_array, columns=encoded_col_names, index=df.index)
    df_encoded = df.drop(columns=[column_to_encode]).copy()
    df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

    return df_encoded

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [4]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    df_encoded = encode_categorical_features(df, column_to_encode='smoker_status')

    X = df_encoded.drop(columns=['disease_outcome'])
    X = X.select_dtypes(exclude=['datetime64[ns]', 'object'])
    X = X.dropna()

    y = df_encoded['disease_outcome']
    y = y.loc[X.index] 


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [5]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    smote = SMOTE(random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    return X_resampled, y_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [7]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return model

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] 

    # 2. Calculate metrics
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_prob),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

    return metrics

## 7. Save Results

Save the evaluation metrics to a text file.

In [8]:
def save_results(metrics: dict):
    os.makedirs('results', exist_ok=True)
    formatted_metrics = [f"{key}: {value}" for key, value in metrics.items()]

    output_file = 'results/results_part3.txt'
    with open(output_file, 'w') as f:
        for line in formatted_metrics:
            f.write(line + '\n')
    print(f"\nMetrics saved to {output_file}")

## 8. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.


In [ ]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    improvements = {}
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
        before = part1_metrics.get(metric, 0)
        after = part3_metrics.get(metric, 0)

        if before == 0:
            improvements[metric] = float('inf') if after > 0 else 0.0  
        else:
            improvements[metric] = ((after - before) / before) * 100

    return improvements

## 9. Main Execution

Run the complete workflow.

In [10]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    save_results(metrics)
    
    # 8. Load Part 1 results for comparison
    import json
    try:
        with open('results/results_part1.txt', 'r') as f:
            part1_metrics = json.load(f)
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 0.8450
precision: 0.3935
recall: 0.8777
f1: 0.5434
auc: 0.9133

Metrics saved to results/results_part3.txt

Model Comparison (improvement percentages):
accuracy: -7.82%
precision: -40.51%
recall: 191.89%
f1: inf%
auc: 42.25%
